In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical

# Constants
IMAGE_SIZE = (128, 128)
DATASET_PATH = '/kaggle/input/indonesian-flora/train'

# Load dataset
def load_dataset(dataset_path):
    images = []
    labels = []
    class_names = os.listdir(dataset_path)
    class_dict = {class_name: idx for idx, class_name in enumerate(class_names)}

    for class_name in class_names:
        class_path = os.path.join(dataset_path, class_name)
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            img = cv2.resize(img, IMAGE_SIZE)
            images.append(img)
            labels.append(class_dict[class_name])
    
    images = np.array(images)
    labels = to_categorical(np.array(labels))
    
    return images, labels, class_names

if __name__ == "__main__":
    images, labels, class_names = load_dataset(DATASET_PATH)
    np.save('images.npy', images)
    np.save('labels.npy', labels)
    np.save('class_names.npy', class_names)


2024-07-22 09:57:28.268091: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-22 09:57:28.268216: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-22 09:57:28.411661: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
libpng warning: iCCP: known incorrect sRGB profile


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import joblib

# Constants
IMAGE_SIZE = (128, 128)

# Load dataset
images = np.load('images.npy')
labels = np.load('labels.npy')
class_names = np.load('class_names.npy', allow_pickle=True)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Train KNN model
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_flat, np.argmax(y_train, axis=1))

# Save KNN model
joblib.dump(knn, 'knn_model.pkl')

# Define and train CNN model
cnn = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(class_names), activation='softmax')
])

cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Save CNN model
cnn.save('cnn_model.h5')


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 20s 386ms/step - accuracy: 0.2961 - loss: 284.5932 - val_accuracy: 0.4515 - val_loss: 1.3743
Epoch 2/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 19s 382ms/step - accuracy: 0.6530 - loss: 1.0233 - val_accuracy: 0.6224 - val_loss: 1.1700
Epoch 3/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 20s 373ms/step - accuracy: 0.8614 - loss: 0.4479 - val_accuracy: 0.6786 - val_loss: 1.0764
Epoch 4/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 21s 380ms/step - accuracy: 0.9658 - loss: 0.1494 - val_accuracy: 0.6786 - val_loss: 1.3087
Epoch 5/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 18s 374ms/step - accuracy: 0.9745 - loss: 0.0870 - val_accuracy: 0.6888 - val_loss: 1.4336
Epoch 6/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 19s 381ms/step - accuracy: 0.9917 - loss: 0.0411 - val_accuracy: 0.6837 - val_loss: 1.3823
Epoch 7/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 21s 387ms/step - accuracy: 0.9920 - loss: 0.0738 - val_accuracy: 0.6811 - val_loss: 1.5539
Epoch 8/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 18s 370ms/step - accuracy: 0.9963 - loss: 0.0313 - val_ac